In [20]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
from collections import Counter
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from imblearn.combine import SMOTEENN
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [21]:
from google.colab import files
files.upload()

!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json


In [22]:
!kaggle datasets download -d kiranmahesh/nslkdd

nslkdd.zip: Skipping, found more recently modified local copy (use --force to force download)


In [23]:
from zipfile import ZipFile
file_name='/content/nslkdd.zip'
with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [24]:
traindata = pd.read_csv('/content/kdd_train.csv')
testdata = pd.read_csv('/content/kdd_test.csv')

In [25]:
data = pd.concat([traindata,testdata])
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [26]:
categ = ['protocol_type','service','flag','labels']
# Encode Categorical Columns
le = LabelEncoder()
data[categ] = data[categ].apply(le.fit_transform)

In [27]:
indexNames = data[ data['labels'] == 35 ].index
# Delete these row indexes from dataFrame
data.drop(indexNames , inplace=True)

In [28]:
X = data.iloc[:,1:41]
Y = data['labels']

In [29]:
scaler = Normalizer().fit(X)
X = scaler.transform(X)

In [30]:
sm = SMOTE(k_neighbors=1)
X_res, y_res = sm.fit_resample(X, Y)

In [31]:
Counter(y_res)

Counter({0: 78587,
         1: 78587,
         2: 78587,
         3: 78587,
         4: 78587,
         5: 78587,
         6: 78587,
         7: 78587,
         8: 78587,
         9: 78587,
         10: 78587,
         11: 78587,
         12: 78587,
         13: 78587,
         14: 78587,
         15: 78587,
         16: 78587,
         17: 78587,
         18: 78587,
         19: 78587,
         20: 78587,
         21: 78587,
         22: 78587,
         23: 78587,
         24: 78587,
         25: 78587,
         26: 78587,
         27: 78587,
         28: 78587,
         29: 78587,
         30: 78587,
         31: 78587,
         32: 78587,
         33: 78587,
         34: 78587,
         36: 78587})

In [32]:
X_train, X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.2)

In [33]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1]))

In [34]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [35]:
model = Sequential()
model.add(Dense(128,input_shape=(40, 1),activation='relu'))  
model.add(Dropout(0.01))
model.add(Flatten())
model.add(Dense(40))
model.add(Activation('softmax'))

In [38]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [39]:
history=model.fit(X_train, Y_train, epochs=100,verbose=1,batch_size=500,validation_data=(X_test, Y_test))

Epoch 1/100
238/238 [==============================] - 5s 10ms/step - loss: 0.9546 - accuracy: 0.7805 - val_loss: 0.4985 - val_accuracy: 0.8650
Epoch 2/100
238/238 [==============================] - 2s 9ms/step - loss: 0.4667 - accuracy: 0.8712 - val_loss: 0.4402 - val_accuracy: 0.8883
Epoch 3/100
238/238 [==============================] - 2s 9ms/step - loss: 0.4258 - accuracy: 0.8869 - val_loss: 0.4091 - val_accuracy: 0.8936
Epoch 4/100
238/238 [==============================] - 2s 9ms/step - loss: 0.3980 - accuracy: 0.8932 - val_loss: 0.3841 - val_accuracy: 0.8952
Epoch 5/100
238/238 [==============================] - 2s 9ms/step - loss: 0.3753 - accuracy: 0.8967 - val_loss: 0.3646 - val_accuracy: 0.8982
Epoch 6/100
182/238 [=====================>........] - ETA: 0s - loss: 0.3571 - accuracy: 0.9003

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()